# Segmenting and Clustering Neighborhoods in Toronto

1. Importing pandas

In [15]:
import pandas as pd

2. Getting postal codes table from wikipedia page

In [16]:
tables = pd.read_html("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
pcodes=tables[0]

3. renaming the Postcode column

In [17]:
pcodes.rename(columns={"Postcode":"PostalCode"}, inplace=True)

In [18]:
pcodes.head()

,PostalCode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


4. Filtering out Borough!="Not assigned"

In [19]:
is_assigned = pcodes["Borough"]!="Not assigned"
pcodes=pcodes[is_assigned]

pcodes.reset_index(inplace=True)
pcodes.drop(["index"],axis=1, inplace=True)
pcodes.head()


/opt/conda/envs/Python36/lib/python3.6/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


5. Concatinating Neighbourhood column for postal codes continaing multiple neighbourhoods

In [20]:
pcodes_concat=pcodes.groupby(['PostalCode','Borough'])['Neighbourhood'].apply(', '.join).reset_index()
pcodes_concat.head()

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


6. Replacing Neighbourhood columns values with Borough values if Neighbourhood = "Not assigned"

In [21]:
pcodes_concat["Neighbourhood"].replace("Not assigned",pcodes_concat["Borough"], inplace=True)


7. Checking the shape of the final dataframe

In [22]:
pcodes_concat.shape

(103, 3)

8. Downloading the CSV with longitude and lattitude data for post codes and renaming the postal code column to match the one from pcodes_concat dataframe

In [23]:
geodata=pd.read_csv("http://cocl.us/Geospatial_data")

In [31]:
geodata.rename(columns={"Postal Code":"PostalCode"}, inplace=True)

In [29]:
pcodes_concat.head()

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


9. joining the the two dataframames 

In [39]:
pcodes_joined = pd.merge(pcodes_concat, geodata, on="PostalCode", how='outer')
pcodes_joined.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


10. confirming if the data joined properly by checking the shape of the new data frame

In [40]:
pcodes_joined.shape

(103, 5)